In [7]:
import pandas as pd
from sqlalchemy import create_engine
import seaborn as sns

In [ ]:
flight_df_path = r"C:\Users\aga_c\Downloads\Pliki_do_Portfolio_Lab (1)\notebooks\flight_df.csv"
flight_df = pd.read_csv(flight_df_path)

In [10]:
username = 'postgres'
password = 'XXXXXXXXXXXXXXX!'
database = 'airlines'
host = 'localhost'
port = 5432 

connection_string = f"postgresql://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

aircraft_df = pd.read_sql_table('aircraft', con=engine)

In [12]:
aircraft_df = aircraft_df.drop(columns=['number_of_seats', 'id'], errors='ignore')
aircraft_df.drop_duplicates(inplace=True)

In [ ]:
aircraft_df_expected_rows = 7364
aircraft_df_expected_columns = set(['tail_num', 'manufacture_year'])

aircraft_df_rows = aircraft_df.shape[0]

diff = aircraft_df_expected_columns.symmetric_difference(set(aircraft_df.columns))
assert aircraft_df_rows == aircraft_df_expected_rows, f'Spodziewano się {aircraft_df_expected_rows} wierszy , otrzymano {aircraft_df_rows} wierszy'

assert diff == set([]), f'Spodziewano się {aircraft_df_expected_columns} kolumn, otrzymano: {aircraft_df_expected_columns} kolumn. Różnica: \n\t{diff}'

In [ ]:
if 'aircraft_df' in locals():
    duplicates_in_tail_num = aircraft_df.groupby('TAIL_NUM')['MANUFACTURE_YEAR'].nunique() > 1

    tail_num_with_multiple_years = duplicates_in_tail_num[duplicates_in_tail_num].index.tolist()

    print(f"Liczba 'tail_num' z więcej niż jednym rokiem produkcji: {len(tail_num_with_multiple_years)}")
else:
    print("Ramka danych 'aircraft_df' nie istnieje.")

tail_num_with_multiple_years[:10] if 'tail_num_with_multiple_years' in locals() else None


In [26]:
aircraft_df['MANUFACTURE_YEAR'] = (
        aircraft_df.groupby('TAIL_NUM')['MANUFACTURE_YEAR']
        .transform('max')
    )

In [ ]:
tmp_flight_df = pd.merge(flight_df, aircraft_df, on='TAIL_NUM', how='inner')

print(tmp_flight_df.head())


In [ ]:
duplicates = tmp_flight_df.duplicated()

print(f"Liczba duplikatów: {duplicates.sum()}")

print(tmp_flight_df[duplicates])

tmp_flight_df = tmp_flight_df.drop_duplicates()

print(f"Liczba duplikatów po usunięciu: {tmp_flight_df.duplicated().sum()}")

In [32]:
flight_df = tmp_flight_df

In [ ]:
flight_df['is_delayed'] = flight_df['DEP_DELAY'] > 0

delays_by_year = flight_df.groupby('MANUFACTURE_YEAR')['is_delayed'].mean().reset_index()

delays_by_year['is_delayed'] = delays_by_year['is_delayed'].round(2)

print(delays_by_year)


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(delays_by_year['MANUFACTURE_YEAR'], delays_by_year['is_delayed'], marker='o', linestyle='-', color='b')

plt.title('Zależność roku produkcji od częstotliwości opóźnień')
plt.xlabel('Rok produkcji')
plt.ylabel('Procent opóźnionych lotów')

plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(delays_by_year ['MANUFACTURE_YEAR'], delays_by_year ['is_delayed'], marker='o', linestyle='-', color='b')

plt.title('Zależność roku produkcji od częstotliwości opóźnień (dla lat z > 10000 lotów)')
plt.xlabel('Rok produkcji')
plt.ylabel('Procent opóźnionych lotów')

plt.grid(True)
plt.show()


In [ ]:
flight_df['MANUFACTURE_YEAR_AGG'] = (flight_df['MANUFACTURE_YEAR'] // 3) * 3

flight_df['is_delayed'] = flight_df['DEP_DELAY'] > 0
flight_delays_by_manufacture_year_agg_df = flight_df.groupby('MANUFACTURE_YEAR_AGG')['is_delayed'].mean().reset_index()

flight_delays_by_manufacture_year_agg_df['is_delayed'] = flight_delays_by_manufacture_year_agg_df['is_delayed'].round(2)

print(flight_delays_by_manufacture_year_agg_df)


In [ ]:
sns.set(style="whitegrid")

plt.figure(figsize=(10, 6))
sns.barplot(data=flight_delays_by_manufacture_year_agg_df, 
            x='MANUFACTURE_YEAR_AGG', 
            y='is_delayed', 
            palette="Blues_d")

plt.title('Częstotliwość opóźnień w zależności od rocznika produkcji samolotu (agregacja co 3 lata)', fontsize=14)
plt.xlabel('Grupa roczników produkcji (co 3 lata)', fontsize=12)
plt.ylabel('Procent opóźnionych lotów', fontsize=12)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
delays_by_year_df = flight_df.groupby('MANUFACTURE_YEAR')['id'].count().reset_index()

top_manufactured_df = delays_by_year_df.sort_values(by='id', ascending=False).head(5)

print(top_manufactured_df)


In [ ]:
file_path = r"C:\Users\aga_c\Downloads\Pliki_do_Portfolio_Lab (1)\notebooks\flight_df_01.csv"

flight_df.to_csv(file_path, index=False)

print(f"Ramka została zapisana w: {file_path}")
